# HW_3 : Bagging and Feature importance

<br><hr style="border: 0; height: 2px; background: red;">

## 1. Data preprocessing과 ML 기법 적용을 위한 package loading

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer   # UCI 저장소의 Breast Cancer Wisconsin (Diagnostic) 데이터셋을 불러오는 함수
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

<br><hr style="border: 0; height: 2px; background: red;">

## 2. Data loading and EDA

In [3]:
# UCI 저장소의 Breast Cancer Wisconsin (Diagnostic) 데이터셋을 loading해서 data 변수공간에 저장
data = load_breast_cancer()

In [4]:
type(data)

sklearn.utils._bunch.Bunch

In [5]:
data.DESCR

'.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n:Number of Instances: 569\n\n:Number of Attributes: 30 numeric, predictive attributes and the class\n\n:Attribute Information:\n    - radius (mean of distances from center to points on the perimeter)\n    - texture (standard deviation of gray-scale values)\n    - perimeter\n    - area\n    - smoothness (local variation in radius lengths)\n    - compactness (perimeter^2 / area - 1.0)\n    - concavity (severity of concave portions of the contour)\n    - concave points (number of concave portions of the contour)\n    - symmetry\n    - fractal dimension ("coastline approximation" - 1)\n\n    The mean, standard error, and "worst" or largest (mean of the three\n    worst/largest values) of these features were computed for each image,\n    resulting in 30 features.  For instance, field 0 is Mean Radius, field\n    10 is Radius SE, field 

In [6]:
X, y = data.data, data.target

`X, y = data.data, data.target` : feature 데이터와 target data를 각각 X, Y로 분리

In [7]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

In [8]:
X.shape, y.shape

((569, 30), (569,))

`X.shape` , `y.shape` : Feature의 행과 열의 수 그리고 target의 행과 (열)의 수 체크!

In [9]:
np.unique(y)   # 0: benign, 1: malignant

array([0, 1])

`np.unique(y)` : target의 레이블 값 확인! (0, 1)만 있으므로 이진분류 task.

In [10]:
feature_names = data.feature_names

In [11]:
feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [12]:
len(feature_names)
#feature_names.shape[0]

30

`feature_names`, `len(feature_names)` : Feature의 이름과 총 feature의 개수를 확인!

<br><hr style="border: 0; height: 2px; background: red;">

## 3. PCA 적용 for 차원축소 -> 주요 정보를 유지한 채 feature의 수 줄이자!

In [13]:
scaler = StandardScaler().fit(X)   
X_scaled = scaler.transform(X)     # features에 표준화를 적용하여 변수 변환

`StandardScaler().fit(X)` : X (Original data)에 평균 0, 표준편차가 1이 되도록 정규화 하는 객체 생성, `.fit(X)`로 X 각 Feature 별로 평균 표준표차 계산 <p>
`scaler.transform(X)` : 위에서 구한 평균, 표준편차로 X의 각 Feature 값을 정규화!

In [14]:
pca = PCA(n_components=0.90, random_state=0) # explained variance의 누적합이 90%가 되는 최소 개수의 주성분을 자동으로 선택  
X_pca = pca.fit_transform(X_scaled)          # random_state=0: 결과 재현성을 위해 seed를 고정

`PCA(n_components = 0.9-, random_state = 0)` : 차원 축소를 위한 PCA 객체 생성 후, <p>
`n_components = 0.9` : 전체 데이터 분산의 90퍼 이상을 설명할 수 있도록 주성분 선택하게 고고 <p>
`random_state = 0`: 시드 고정 <p>
`pca.fit_transform(X_scaled)` : X_sceled의 공분산을 파악하여 분산이 가장 큰 방향 즉 주성분 벡터를 계산 -> `transform()` -> 데이터를 전에 구한 주성분 방향으로 투영하여 저차원으로 변환.


In [15]:
X_pca.shape

(569, 7)

-> 행: 샘플의 수, 열: 선택된 주성분의 수

In [16]:
n_pc = X_pca.shape[1]
n_pc

7

<br><hr style="border: 0; height: 2px; background: red;">

## 4. Grid Serach for Bagging and Model fitting

In [17]:
### OOB(Out-of-Bag) 스코어를 기준으로 Bagging 분류기의 주요 하이퍼파라미터를 Grid Search하기 위해 준비하는 부분 ###
results = []                        
param_grid = {                      ## key: Bagging Classifier의 주요 하이퍼파라미터들
    'n_estimators': [50, 100, 200],  # n_estimators: 앙상블에 사용할 트리의 개수   
    'max_samples': [0.7, 1.0],       # max_samples : 각 트리를 학습시킬 때 사용할 샘플의 비율
    'max_features': [0.7, 1.0]       # max_features: 각 트리 분할 시 고려할 특성의 비율
}                                   ## 하이퍼파라미터들의 조합의 수 = 3*2*2 = 12개

In [18]:
## param_grid에 정의된 하이퍼파라미터 조합을 전부 test해 보고
## 각 조합별로 OOB(Out-of-Bag) 스코어를 계산해
## results 리스트에 저장하는 그리드 탐색(grid search)을 수행한다

for n_est in param_grid['n_estimators']:                 # 'n_estimators': [50, 100, 200] 
    for max_s in param_grid['max_samples']:              # 'max_samples': [0.7, 1.0]
        for max_f in param_grid['max_features']:         # 'max_features': [0.7, 1.0]
            clf = BaggingClassifier(                    ## BaggingClassifier를 생성한다
                estimator=DecisionTreeClassifier(),      # 기본 학습기로 DecisionTreeClassifier를 사용한다
                n_estimators=n_est,
                max_samples=max_s,
                max_features=max_f,
                oob_score=True,                          # OOB 샘플을 사용하여 성능을 평가한다
                random_state=0,
                n_jobs=-1                                # 가능한 모든 CPU 코어를 활용해 병렬 학습한다
            )
            clf.fit(X_pca, y)
            results.append({
                'n_estimators': n_est,
                'max_samples': max_s,
                'max_features': max_f,
                'oob_score': clf.oob_score_               # 학습이 끝나면 clf.oob_score_ 속성에 저장된 OOB 정확도를 읽어온다
            })

위에서 설정한 `param_grid`는 총 12가지 조합(3×2×2)에 대해 `BaggingClassifier`의 하이퍼파라미터 설정을 만들어둔 것이다!
이제 그 각각의 조합에 대해 실제로 모델을 학습(fit) 하고, OOB 점수(검증 정확도) 를 계산해서 results 리스트에 저장하는 코드가 바로 위의 셀에서 실행한돠.

바로 위의 셀에서는:

1. 3중 for-loop를 통해 `n_estimators`, `max_samples`, `max_features`의 모든 조합을 순회하면서

2. 각 조합마다 `BaggingClassifier` 모델을 생성하고, X_pca, y에 대해 학습시킨ㄷ,

3. `oob_score=True`로 설정했기 때문에, 학습에 사용되지 않은 샘플(Out-of-Bag)을 이용하여 모델 성능을 자동으로 평가한다.

> * oob_score(bool type parameter): 생성자 인자로 주어지는 설정값. 학습할 때 OOB 점수를 계산할 것인지 여부를 TRUE 또는 FALSE로 설정한다
> * oob_socre_(float type attribute): fit() 호출 뒤, 실제로 계산된 OOB 점수를 담은 결과값. 분류 task: 정확도. 회귀 task: 수정된 결정계수.

4. 최종적으로 각 조합과 대응되는 OOB 스코어를 `results` 리스트에 딕셔너리 형태로 저장한다.

In [19]:
vars(clf).keys()

dict_keys(['estimator', 'n_estimators', 'estimator_params', 'max_samples', 'max_features', 'bootstrap', 'bootstrap_features', 'oob_score', 'warm_start', 'n_jobs', 'random_state', 'verbose', 'n_features_in_', '_n_samples', 'classes_', 'n_classes_', 'estimator_', '_max_samples', '_max_features', 'estimators_', 'estimators_features_', '_seeds', 'oob_decision_function_', 'oob_score_'])

`vars(clf).keys()`는 ㅔython의 내장 함수 `vars()`를 사용해 객체 clf의 attribute들을 딕셔너리 형태로 확인하 key만 가져온다.

여기서 clf는 `BaggingClassifier` instance이므로 이 코드는 `BaggingClassifier` 객체가 내부적으로 어떤 attribute를 갖고 있는지 확인해준다,

In [20]:
type(results)

list

In [21]:
results_df = pd.DataFrame(results)
results_df

,n_estimators,max_samples,max_features,oob_score
0,50,0.7,0.7,0.940246
1,50,0.7,1.0,0.945518
2,50,1.0,0.7,0.943761
3,50,1.0,1.0,0.949033
4,100,0.7,0.7,0.956063
5,100,0.7,1.0,0.952548
6,100,1.0,0.7,0.956063
7,100,1.0,1.0,0.947276
8,200,0.7,0.7,0.950791
9,200,0.7,1.0,0.950791


-> 12개 하이퍼파라미터들의 조합에 대해 4개의 딕셔너리 key의 value를 보여준다

In [22]:
type(results_df['oob_score'])

pandas.core.series.Series

In [23]:
best_idx = results_df['oob_score'].idxmax() # 이 Series에서 가장 큰 값을 갖는 인덱스를 반환한다
best_idx

10

`idxmax()` method를 이용하여 가장 큰 값을 갖는 인덱스를 찾아준다.

`idxmin()`도 있다..!

In [24]:
#
best_params = results_df.loc[best_idx].to_dict()
best_params

{'n_estimators': 200.0,
 'max_samples': 1.0,
 'max_features': 0.7,
 'oob_score': 0.9578207381370826}

-> results_df에서 가장 높은 OOB 스코어를 기록한 행(row)을 딕셔너리 형태로 뽑아서 best_params에 저장한 후 출력한다

아래의 코드는 앞서 Grid Search로 찾은 최적의 하이퍼파라미터 조합(best_params)을 이용해서 최종적으로 사용할 BaggingClassifier 모델을 재생성하는 단계이다.

In [25]:
# Grid Search로 찾아낸 최적 하이퍼파라미터를 이용해 최종 BaggingClassifier 모델을 다시 생성한다
best_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=int(best_params['n_estimators']),
    max_samples=best_params['max_samples'],
    max_features=best_params['max_features'],
    oob_score=True,
    random_state=0,
    n_jobs=-1
)

`estimator=DecisionTreeClassifier()` : 배깅이 사용할 기본 모델로 의사결정트리를 지정하는 것 <p>
`n_estimators=int(best_params['n_estimators'])` : 앙상블 내에 몇 개의 트리를 사용할지를 설정 <p>
`max_samples=best_params['max_samples']` : 각 트리를 학습시킬 때 전체 데이터 중 몇 %의 샘플을 사용할지를 의미 <p>
`max_features=best_params['max_features']` : 각 트리가 분할을 고려할 때 사용할 특성의 비율을 의미 <p>
 `oob_score=True는 OOB(Out-of-Bag)` :  샘플을 사용해서 모델의 일반화 성능을 추정하게 해 준다 <p>
 `random_state=0` : 시드를 고정 <p>
`n_jobs=-1` : 모든 CPU 코어를 사용해 병렬 처리를 수행하게 하여 학습 속도를 up <p>

In [26]:
best_clf.fit(X_pca, y)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_features=0.7,
                  n_estimators=200, n_jobs=-1, oob_score=True, random_state=0)

`best_clf.fit(X_pca, y)` : 위에서 생성한 최종 BaggingClassifier 모델(best_clf)을 PCA로 차원을 축소한 입력 데이터 X_pca와 그에 대응되는 정답 레이블 y를 사용해서 학습!

<br><hr style="border: 0; height: 2px; background: red;">

## 5. Step for compute Feature importance.

In [27]:
### PC별 MDI 중요도 복원
pc_importances = np.zeros(n_pc) # n_pc: PCA로 축소된 후 남은 주성분의 수. PC별 누적 중요도를 저장할 배열을 만들고, 0으로 값 초기화
M = len(best_clf.estimators_)   # best_clf.estimators_: 학습된 트리 인스턴스들의 리스트. 그 리스트의 길이 M: 앙상블에 포함된 트리의 수

위의 코도는 PCA로 축소된 주성분(PC) 공간에서 각 축의 중요도를 계산하기 위한 단계이다.

`pc_importances = np.zeros(n_pc)` : PCA 변환 후 남아 있는 주성분의 수만큼의 길이를 가진 배열을 만들고, 모든 값을 0으로 초기화, 이 배열은 이후 각 트리의 feature importance를 누적합산하여, 각 주성분(PC)의 평균적인 중요도를 계산하기 위해 사용.

`M = len(best_clf.estimators_)` : 학습된 배깅 모델(best_clf)에 포함된 트리의 개수를 가져옴.

`best_clf.estimators_` : 실제 학습이 끝난 후 내부적으로 생성된 결정트리 인스턴스들의 리스트이고, 이 리스트의 길이 M은 앙상블 안에 몇 개의 트리가 포함되어 있는지를 알려줌.

In [28]:
for tree, feats in zip(best_clf.estimators_, best_clf.estimators_features_):
    imp = tree.feature_importances_   # tree.feature_importances_: 적합된 트리가 len(feats)개의 축에 대해 계산한 MDI 중요도 벡터
                                      # 벡터 길이: len(feats). imp[i]는 feats[i]번째 주성분 축이 그 트리에서 얼마나 불순도 감소에 기여했는지
    pc_importances[feats] += imp      # feats에 대응하는 위치(pc_importances[feats])에 imp를 더해 누적한다

배깅 앙상블에 포함된 각 트리의 MDI 기준 중요도를 PCA 주성분 축별로 누적 계산하는 단계!!!

`for tree, feats in zip(best_clf.estimators_, best_clf.estimators_features_)`:

각 학습된 트리 객체(tree)와, 해당 트리가 분할 시 사용한 PCA 축의 인덱스 리스트(feats)를 동시에 반복.

`imp = tree.feature_importances_` : 해당 트리에서 계산된 MDI 기준의 feature 중요도 벡터를 가져옴.

> 이 벡터는 길이가 `len(feats)`이고 값들은 해당 PCA 축이 그 트리에서 얼마나 Gini impurity를 감소시키는 데 기여했는지를 보여줌,

`pc_importances[feats] += imp` : 트리의 중요도 imp를 feats에 대응되는 위치의 pc_importances 배열에 더해서 누적..!



In [29]:
pc_importances /= M
# 트리 수 M으로 나누어, 모든 트리에 걸친 평균 중요도로 정규화한다
# pc_importance[k]: 전체 앙상블에서 k번째 주성분이 받은 평균 MDI 중요도

이제 모든 트리에 대해 중요도가 누적되었으니 평균 가보자!

PCA 축이 전체 앙상블에서 받은 평균적인 중요도를 얻을 수 있다..

In [30]:
## PCA 축 단위로 계산된 평균 중요도(pc_importances)를 다시 원본 변수(feature) 수준으로 되돌려, 각 원본 변수별 중요도를 근사하는 과정
loadings = np.abs(pca.components_.T) # (n_orig_features, n_pc) => 각 원본 변수가 어떤 주성분 축들과 얼마나 연관되는지가 열 방향으로 대응  
                                     # loadings[i,k]: 원본 변수 i가 주성분 k에 기여하는 정도
# pca.components_: pc의 로딩(loading) 벡터를 행 단위로 담고 있다
# pca.components_의 shape = (n_pc, n_orig_features)

PCA의 로딩 행렬에서 절댓값만 취한 형태로, (원본 변수 개수, 주성분 개수) 크기를 갖는 행렬이고

loadings[i, k]는 원본 변수 i가 주성분 k에 얼마나 기여했는지를 나타낸다.

이걸 통해서 “PCA 축 단위 중요도”를 “원래 변수 단위 중요도”로 역산 가능.

In [31]:
orig_importances = loadings.dot(pc_importances) # 행렬곱(dot product)
# 각 원본 변수 i가 속한 주성분 축들에 나뉘어 있는 중요도를 로딩 비율에 따라 합산해서, 원본 변수 수준의 근사 MDI 중요도를 얻는다

PCA 축별 중요도(pc_importances)와 로딩 행렬을 행렬곱(dot product)해서

각 원본 변수 i가 얼마나 중요한 역할을 했는지를 근사하는 중요도 벡터 `orig_importance`를 만듦.

-> 주성분으로 분산된 중요도를 로딩 비율만큼 원래 변수로 다시 재구성

In [32]:
orig_imp_df = pd.DataFrame({
    'feature': feature_names,
    'importance': orig_importances
}).sort_values('importance', ascending=False).reset_index(drop=True) # importance 열을 기준으로 내림차순 정렬
## reset_index(drop=True)
# 정렬 과정에서 뒤섞인 인덱스(원래 행 번호)를 0,1,2… 순서로 새로 매긴다
# drop=True 옵션으로 원래 인덱스 열은 버린다

feature anme과 importance를 한 Dataframe에 정리하고, 중요도 기준으로 내림차순 정렬

그리고 `.reset_index(drop=True)`를 통해 정렬로 섞인 기존 인덱스를 새롭게 0부터 다시 부여

In [33]:
# best_params 딕셔너리에 저장된 최적 하이퍼파라미터 이름과 값 쌍을 pandas DataFrame으로 바꾼다
param_df = pd.DataFrame(
    best_params.items(),            
    columns=["Parameter","Value"] # 2개의 열 이름을 지정한다
)

In [34]:
print('Best hyperparameters (OOB 기준)')
print()
param_df

Best hyperparameters (OOB 기준)



,Parameter,Value
0,n_estimators,200.000000
1,max_samples,1.000000
2,max_features,0.700000
3,oob_score,0.957821


In [35]:
print('OOB Hyperparameter Results')
print()
results_df

OOB Hyperparameter Results



,n_estimators,max_samples,max_features,oob_score
0,50,0.7,0.7,0.940246
1,50,0.7,1.0,0.945518
2,50,1.0,0.7,0.943761
3,50,1.0,1.0,0.949033
4,100,0.7,0.7,0.956063
5,100,0.7,1.0,0.952548
6,100,1.0,0.7,0.956063
7,100,1.0,1.0,0.947276
8,200,0.7,0.7,0.950791
9,200,0.7,1.0,0.950791


In [36]:
print('Original Feature Importances (MDI+PCA)')
print()
orig_imp_df

Original Feature Importances (MDI+PCA)



,feature,importance
0,area error,0.186349
1,radius error,0.185717
2,perimeter error,0.181980
3,concave points error,0.178471
4,worst compactness,0.174915
5,mean smoothness,0.170557
6,worst concavity,0.170186
7,worst smoothness,0.166126
8,worst fractal dimension,0.164722
9,worst concave points,0.163172


<br><hr style="border: 0; height: 2px; background: red;">

## 최종 요약

이 노트북은 UCI의 유방암 진단 데이터를 활용하여 다음 과정을 단계별로 수행:

1. **Preprocessing** : 원본 데이터를 표준화하여 각 특성의 단위 차이를 제거
2. **Dim-reduction** : PCA를 이용해 전체 분산의 90%를 설명할 수 있도록 feature 수를 줄인다.
3. **Model training (Bagging + Decision Tree)** :
   - 배깅(Bagging) 기반 앙상블 분류기를 구성하고
   - OOB(Out-of-Bag) 점수를 기준으로 주요 하이퍼파라미터 조합을 수동으로 탐삭.
4. **Compute Feature importance** : 최적의 조합으로 모델을 재학습한 ->
   - PCA 주성분 기준으로 계산된 중요도를
   - 로딩 행렬을 활용해 원본 변수 수준의 중요도로 역산.

-> 이 코드는 "PCA로 차원 축소된 공간에서 배깅 앙상블 모델을 학습하고, 그 결과를 다시 원래 feature의 중요도로 해석"를 구현